In [ ]:
import helpers
import numpy as np
from collections import defaultdict
import re

data = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""

with open("input_10.txt", 'r') as f:
    data = f.read().rstrip()

def parse_line(line):
    # []
    bracket_str = re.search(r"\[(.*?)\]", line).group(1)
    first = tuple(1 if c == '#' else 0 for c in bracket_str)

    # ()
    paren_parts = re.findall(r"\((.*?)\)", line)
    second = [tuple(map(int, p.split(','))) for p in paren_parts]

    # {}
    curly_str = re.search(r"\{(.*?)\}", line).group(1)
    third = tuple(map(int, curly_str.split(',')))

    return first, second, third

data = [
    parse_line(line)
    for line in data.splitlines()
    if line.strip()
]

In [ ]:
#intuition, we can solve it with integer programming, I let ChatGPT write the base ILP code 
#because ain't nobody got time for figuring out how ILP Python code has to be written
#next we just have to programtically set the variables to be the different presses combinations.

#e.g. for moves [(1,), (0,), (0,1,2)] 
#to get to (10, 5, 2) we minimize (x,y,z) s.t. x * (0,1,0) + y * (1,0,0) + z * (1,1,1) == (10, 5, 2)
#Linear Programming can be finicky when working with floats I experienced last year
#Luckily, (since it's integers this time?) we have no issues and the answers are correct

import pulp
ans = 0
def generate_generators(moves, state):
    generators = {}
    for i, move in enumerate(moves):
        generators[f"x_{i}"] = toggle_machine((0,) * len(state), move)
    return generators

for state, moves,target in data:
    generators = generate_generators(moves, state)

    # model
    prob = pulp.LpProblem("MinGenerators", pulp.LpMinimize)

    # integer vars >= 0
    vars_ = {
        name: pulp.LpVariable(name, lowBound=0, cat=pulp.LpInteger)
        for name in generators
    }

    # objective: minimize sum of coefficients
    prob += pulp.lpSum(vars_.values())

    # constraints: exact hit here; swap == with >= / <= as desired
    dim = len(target)
    for d in range(dim):
        prob += (
            pulp.lpSum(vars_[name] * generators[name][d] for name in generators)
            == target[d]
        )

    # solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    solution = {name: int(vars_[name].value()) for name in generators}
    total = sum(solution.values())

    #print("solution:", solution, "total:", total)
    ans += total
ans

14999